# Importation des bibliothèques nécessaires
Importer les bibliothèques nécessaires comme pandas, numpy, matplotlib, et les modules spécifiques pour la modélisation des séries temporelles.

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# Chargement et préparation des données
Charger les données depuis un fichier CSV, trier par date, et définir l'index sur la colonne de date.

In [2]:
# Chargement et préparation des données
data = pd.read_csv("monthly_market_arrivals.csv")
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date')
data.set_index('date', inplace=True)

data.head()

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city
date,,,,,,,,,
1996-01-01,LASALGAON(MS),January,1996,225063,160,257,226,MS,LASALGAON
1996-02-01,LASALGAON(MS),February,1996,196164,133,229,186,MS,LASALGAON
1996-03-01,LASALGAON(MS),March,1996,178992,155,274,243,MS,LASALGAON
1996-04-01,LASALGAON(MS),April,1996,192592,136,279,254,MS,LASALGAON
1996-05-01,LASALGAON(MS),May,1996,237574,154,312,269,MS,LASALGAON


# Décomposition de la série temporelle
Utiliser une bibliothèque comme statsmodels pour décomposer la série temporelle en tendance, saisonnalité et résidus.

In [3]:
# Décomposition de la série temporelle
from statsmodels.tsa.seasonal import seasonal_decompose

ts_data = data.loc[data.city == "BANGALORE", 'onion_sales']
decomposition = seasonal_decompose(ts_data, model='additive', period=12)

decomposition.plot()
plt.show()

ModuleNotFoundError: No module named 'statsmodels'

# Création des variables décalées (lags)
Créer des colonnes pour les variables décalées de 1 à 7 mois pour capturer les dépendances temporelles.

In [ ]:
# Création des variables décalées (lags)
for lag in range(1, 8):
    ts_data[f'lag_{lag}'] = ts_data.shift(lag)

ts_data.dropna(inplace=True)
ts_data.head()

# Division des données en ensembles d'entraînement et de test
Diviser les données en ensembles d'entraînement (85%) et de test (15%) en utilisant scikit-learn.

In [ ]:
# Division des données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

X = ts_data.drop(columns=['onion_sales'])
y = ts_data['onion_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)

X_train.shape, X_test.shape

# Régression linéaire sur les données d'entraînement
Effectuer une régression linéaire sur les données d'entraînement et calculer le R² sur les données de test. Visualiser les valeurs prédites et réelles.

In [ ]:
# Régression linéaire sur les données d'entraînement
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

plt.plot(y_test.index, y_test, label='Valeurs réelles')
plt.plot(y_test.index, y_pred, label='Valeurs prédites')
plt.legend()
plt.title(f"R²: {r2:.2f}")
plt.show()

# Ajout de variables agrégées et création d'une fonction de prétraitement
Créer une fonction pour ajouter des variables agrégées comme les moyennes mensuelles et préparer les données en une seule étape.